# LinkedIn Post Analyser

This Python code reads a .csv file stored in Google Drive, containing 5 LinkedIn posts. Then sends its content to a gpt model using OpenAi API and prompts the model for analysis and improvements. Then receives the output from LLM and saves it as a new file on Google Drive.

1. Install required libraries

In [1]:
!pip install openai pandas

2. Import all required workflows into the notebook

In [2]:
import os
import pandas as pd
from openai import OpenAI
from google.colab import userdata, drive
from IPython.display import display, Markdown

3. Defining a function for the prompt for a LLM. Prompt includes two parts: LinkedIn post analysis and creation of an improved version.

In [3]:
def get_chatgpt_prompt(linkedin_post):
  evaluation_prompt = f"""
Please critically analyze the following LinkedIn post and then provide an improved version of it.
Ensure the improved version is in the same language as the original post.

---
Original LinkedIn Post:
{linkedin_post}
---

First, provide your analysis based on the following criteria:

### Analysis:
1.  **Content Quality:** Is the post well-written, clear, professional, and free of grammatical errors or typos?
2.  **Reader Engagement:** How likely is this post to engage its readers? Does it invite interaction (e.g., questions, calls to action), or is it highly relatable?
3.  **Value Provided:** Does the post offer genuine value, actionable insights, solutions, or thought-provoking content to the reader?
4.  **Length Analysis:** Is the post of an optimal length for LinkedIn (e.g., concise enough to hold attention, but long enough to convey the message)?
5.  **LinkedIn Formatting & Readability:** Does the post utilize formatting best practices for LinkedIn (e.g., use of short paragraphs, bullet points, emojis, line breaks, hashtags, bold/italic text if applicable) to enhance readability and visual appeal?

Provide specific feedback and suggestions where appropriate for each analysis point.

Second, provide an improved version of the post. This improved version should address the issues identified in the analysis and incorporate LinkedIn best practices.

### Improved Post:
"""
  return evaluation_prompt

4. Defining a function to get a response from the LLM. System role: expert in social media marketing and content analysis.

In [4]:
def get_insights_from_chatgpt(client, linkedin_post):
  prompt = get_chatgpt_prompt(linkedin_post)
  try:
    response = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {"role": "system", "content": "You are an expert in social media marketing and content analysis. Provide clear, actionable feedback and rewrite posts concisely."},
        {"role": "user", "content": prompt}
      ],
      temperature = 0.5,
      max_tokens = 1000,
      top_p = 0.8
    )
    full_response_content = response.choices[0].message.content

    analysis_start_tag = "### Analysis:"
    improved_post_start_tag = "### Improved Post:"

    analysis_text = "Analysis could not be parsed."
    improved_post_text = "Improved post could not be parsed."

    if analysis_start_tag in full_response_content and improved_post_start_tag in full_response_content:
        analysis_section = full_response_content.split(improved_post_start_tag)[0]
        improved_post_section = full_response_content.split(improved_post_start_tag)[1]

        if analysis_section.startswith(analysis_start_tag):
            analysis_text = analysis_section[len(analysis_start_tag):].strip()
        else:
            analysis_text = analysis_section.strip()

        improved_post_text = improved_post_section.strip()
    else:
        analysis_text = "Warning: Expected headings '### Analysis:' and '### Improved Post:' not found. Full response:\n" + full_response_content
        improved_post_text = "Parsing failed: Improved post not clearly separated."

    return analysis_text, improved_post_text
  except Exception as e:
    return f"Error: {e}", None

5. Connect to Google Drive to access files.

In [5]:
def mount_google_drive():
  print("Mounting Google Drive...")
  try:
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
    return True
  except Exception as e:
    print(f"Error mounting Google Drive: {e}")
    print("Please ensure you've authorized Colab to access your Google Drive.")
    return False

6. Access and read a .csv file on Google Drive.

In [6]:
def load_linkedin_posts(csv_file_path):
  try:
    df = pd.read_csv(csv_file_path, sep=';')
    if 'post_content' not in df.columns:
        print(f"Error: CSV file must contain a column named 'post_content'. Found columns: {df.columns.tolist()}")
        print("Please ensure your CSV has a 'post_content' column or update the code to match your column name.")
        return None
    linkedin_posts = df['post_content'].tolist()
    print(f"Successfully loaded {len(linkedin_posts)} posts from '{csv_file_path}'.")
    return linkedin_posts
  except FileNotFoundError:
    print(f"Error: CSV file not found at '{csv_file_path}'. Please check the path.")
    return None
  except pd.errors.EmptyDataError:
    print(f"Error: CSV file at '{csv_file_path}' is empty.")
    return None
  except Exception as e:
    print(f"Error reading CSV file: {e}")
    return None

7. Connect to OpenAI API using the API key saved in Colab Secrets.

In [7]:
def get_openai_client():
  try:
    openai_api_key = userdata.get("OPENAI_API_KEY")
  except Exception as e:
    print(f"Error retrieving API key from Colab Secrets: {e}")
    print("Please ensure 'OPENAI_API_KEY' is added to Colab Secrets and 'Notebook access' is enabled.")
    return None

  if not openai_api_key:
    print("Error: OPENAI_API_KEY secret not found or empty in Colab Secrets.")
    print("Please ensure it's added and 'Notebook access' is enabled.")
    return None

  try:
    client = OpenAI(api_key=openai_api_key)
    return client
  except Exception as e:
    print(f"Error initializing OpenAI client: {e}")
    return None

8. Save the LLM output back on Google Drive.

In [8]:
def save_output_to_drive(all_outputs):
  """
  Saves the compiled analysis output to a Markdown file on Google Drive.
  """
  output_drive_path = input("\nPlease enter the desired path to save the output file on Google Drive (e.g., /content/drive/My Drive/linkedin_analysis_results.md): ").strip()

  if not output_drive_path:
    output_drive_path = "/content/drive/My Drive/linkedin_analysis_results.md"
    print(f"No output path provided. Saving to default: {output_drive_path}")
  try:
    with open(output_drive_path, "w", encoding="utf-8") as f:
      f.write("\n".join(all_outputs))
    print(f"\nAll analysis and improved posts successfully saved to: '{output_drive_path}'")
    print("You can find this file in your Google Drive and download it from there.")
    return True
  except Exception as e:
    print(f"Error saving output file to Google Drive: {e}")
    return False

9. Definition of the main function to orchestrate the LinkedIn post analysis and improvement workflow.

In [9]:
def main():
  if not mount_google_drive():
    return

  csv_file_path = input("\nPlease enter the path to your CSV file on Google Drive (e.g., /content/drive/My Drive/linkedin_posts.csv): ").strip()
  if not csv_file_path:
      print("No CSV file path provided. Exiting.")
      return

  linkedin_posts = load_linkedin_posts(csv_file_path)
  if linkedin_posts is None:
    return

  client = get_openai_client()
  if client is None:
    return

  all_outputs = []
  print("\n--- Starting LinkedIn Post Analysis and Improvement ---")

  for i, original_post in enumerate(linkedin_posts):
    print(f"\nProcessing Post {i+1}/{len(linkedin_posts)}...")

    if not isinstance(original_post, str):
        original_post = str(original_post)

    analysis, improved_post = get_insights_from_chatgpt(client, original_post)

    current_post_output = f"""
--- Post {i+1} ---
### Original Post:
{original_post}

### ChatGPT's Analysis:
{analysis}

### Improved Version:
{improved_post}
"""
    all_outputs.append(current_post_output)
    display(Markdown(current_post_output))

  print("\n--- Analysis Complete ---")
  save_output_to_drive(all_outputs)

10. Run the code

In [10]:
main()

Mounting Google Drive...
Mounted at /content/drive
Google Drive mounted successfully.

Please enter the path to your CSV file on Google Drive (e.g., /content/drive/My Drive/linkedin_posts.csv): /content/drive/My Drive/linkedin_posts.csv
Successfully loaded 5 posts from '/content/drive/My Drive/linkedin_posts.csv'.

--- Starting LinkedIn Post Analysis and Improvement ---

Processing Post 1/5...



--- Post 1 ---
### Original Post:
«During Pride Month, Kering proudly celebrated the vibrant LGBTQIA+ community across our Group.

This year, Kering and our Houses joined Pride parades in Paris, Milan, Rome, Tokyo, London, Mexico City, and cities across the United States.

We also partnered with The Center, a leading LGBTQIA+ organization, to host special events in New York and Los Angeles.

At Kering, Pride doesn’t end in June. Through ALL, our internal LGBTQIA+ network, we champion inclusion year-round with open conversations, dedicated activities and learning modules.

Learn more about how Kering empowers talent:
https://lnkd.in/eDWxBxhv

hashtag
#KeringForTalent»

### ChatGPT's Analysis:
1. **Content Quality:** 
   - The post is generally well-written and professional, with no grammatical errors. However, it could benefit from a more engaging tone to resonate with the audience.

2. **Reader Engagement:** 
   - While the post shares important information, it lacks direct engagement strategies. There are no questions posed or prompts for readers to share their experiences or thoughts, which could enhance interaction.

3. **Value Provided:** 
   - The post provides valuable insights into Kering’s initiatives for the LGBTQIA+ community. However, it could include specific examples of the impact of these initiatives or invite readers to participate in ongoing discussions.

4. **Length Analysis:** 
   - The post is a bit lengthy and could be more concise. Shortening some sentences and removing less critical details would help maintain reader attention.

5. **LinkedIn Formatting & Readability:** 
   - The post could improve its readability by using line breaks for better structure. Adding bullet points for the cities where Kering participated in Pride events would make the information easier to digest.

### Improved Version:
🌈 **Celebrating Pride at Kering** 🌈

During Pride Month, we proudly celebrated the vibrant LGBTQIA+ community across our Group! 

This year, Kering and our Houses participated in Pride parades in:
- Paris
- Milan
- Rome
- Tokyo
- London
- Mexico City
- Various U.S. cities

We also partnered with The Center, a leading LGBTQIA+ organization, to host special events in New York and Los Angeles.

At Kering, our commitment to inclusion goes beyond June! Through ALL, our internal LGBTQIA+ network, we foster open conversations and activities year-round.

👉 **Join us in championing diversity!** Learn more about how Kering empowers talent: [https://lnkd.in/eDWxBxhv](https://lnkd.in/eDWxBxhv)

#KeringForTalent #Pride #Inclusion #Diversity #LGBTQIA+ 

---

This improved version enhances engagement, readability, and clarity while maintaining the original message.



Processing Post 2/5...



--- Post 2 ---
### Original Post:
«Šią vasarą kviečiame ne tik dėvėti, bet ir išgyventi vieną garsiausių lietuviškų legendų - Eglės Žalčių karalienės mitą.

Vasaros kolekcijos pasakojimą kūrėme kartu su dviem garsiomis Lietuvos kūrėjomis - Lina Vyšniauskaite (Lina Disciplina) ir Jolita Vaitkutė. Kartu bandėme atsakyti į klausimą: ką reiškia būti lietuviu šiandien? Atsakymas slypi Eglės žalčių karalienės mite – moderniai perteiktame per grafiką, formas, simbolius ir natūralias medžiagas.

🎨 Lina Vyšniauskaitė (Lina Disciplina) kolekcijoje atnešė aiškų, grafinį ir disciplinuotą braižą. Jos Eglė – tarsi iškalta iš linijų, tvirta, moderni, bet su giliu žvilgsniu.
🌿 Jolita Vaitkutė rėmėsi natūralumo ir gamtos pojūčiu. Ji dirbo su tikromis medžiagomis – žalio šilko atspalviais, žolynais, tošimi, tarsi jungdama tikrą Lietuvos mišką su tekstile.

Ši AUDIMAS kolekcija – tai daugiau nei tik vasaros drabužiai. Tai priminimas apie ryšį su tuo, kas pastovu: gamta, tikrumu ir tapatybe, kurią nešiojamės viduje – dabar ir visada.

hashtag
#AUDIMAS 
hashtag
#VasarosKolekcija»

### ChatGPT's Analysis:
1. **Content Quality:** 
   - The post is well-written and free of grammatical errors. It conveys a clear message about the summer collection inspired by a Lithuanian legend. However, it could benefit from a more engaging opening line to capture attention immediately.

2. **Reader Engagement:**
   - While the post shares an interesting narrative, it lacks a direct call to action or an invitation for reader interaction. Posing a question or encouraging readers to share their thoughts on the legend could enhance engagement.

3. **Value Provided:**
   - The post provides cultural context and insight into the collection’s inspiration, which adds value. However, it could offer more actionable insights, such as where to purchase the collection or how to experience it.

4. **Length Analysis:**
   - The length is appropriate for LinkedIn, but the message could be more concise. Some sentences are lengthy and could be streamlined for better readability.

5. **LinkedIn Formatting & Readability:**
   - The post uses emojis effectively, but it could benefit from shorter paragraphs or bullet points to break up the text. This would enhance readability and make key points stand out.

### Improved Version:
🌟 **Šią vasarą kviečiame išgyventi Eglės Žalčių karalienės legendą su mūsų nauja vasaros kolekcija!**

Kartu su talentingomis kūrėjomis Lina Vyšniauskaite (Lina Disciplina) ir Jolita Vaitkute, atsakome į klausimą: **ką reiškia būti lietuviu šiandien?** 

✨ **Lina Vyšniauskaitė (Lina Disciplina):** 
- Sukūrė modernią Eglės interpretaciją, kuri simbolizuoja stiprybę ir gylį.

🌿 **Jolita Vaitkutė:** 
- Naudojo natūralias medžiagas, jungdama Lietuvos mišką su tekstile.

Ši AUDIMAS kolekcija yra daugiau nei drabužiai – tai ryšys su gamta ir mūsų tapatybe. 

🔗 **Norite sužinoti daugiau? Pasidalinkite savo mintimis apie Eglę arba aplankykite mūsų svetainę!**

#AUDIMAS #VasarosKolekcija



Processing Post 3/5...


...


Processing Post 4/5...



--- Post 4 ---
### Original Post:
«Praėjusią savaitę užbaigėme Klaipėdoje, kur vyko kartu su Baltic Sandbox organizuojamos 3 dalių dirbtuvės ir patirčių dalybos su startuolių kūrėjais ir verslo lyderiais! 🙌

✔️ BSV Ventures generalinė partnerė Sandra Golbreich pasakojo, kaip supaprastinti sprendimų priėmimą, greičiau prisitaikyti prie pokyčių ir transformuoti tradicines verslo struktūras
✔️ SPARKLY.HR įkūrėjas ir vadovas, verslo patarėjas ir mentorius Mikk Orglaan padėjo geriau suprasti, kaip veikia startuoliai, jų kitokį požiūrį į marketingą ir kaip efektyviai planuoti augimą
✔️ Apie savo klaidas atvirai pasakojo (ne)sėkmių patirtimis versle bei gyvenime dalinosi: Beatos Virtuvė vadovė Odeta Bloziene; CHAZZ CHIPS bendraįkūrėjas Žilvinas Kulvinskis; Stand UP Advertising agency įkūrėja Inesa Budgine 🆙.

Dėkojame renginio partneriams: Baltic Sandbox ir BSV Ventures.

Ačiū tariame renginio rėmėjams: UAB Švyturys - Utenos alus9 (nealkoholinis), Aromatic •89•, Angel SPA; IQ_zurnalas, S. Jokužio leidykla - spaustuvė, Giftika, UAB SAKALAS, kurie dovanoja Elephant ir Merci produkciją, Tubinas Hotels, Mama reklama, Klaipėdos m. savivaldybė/Klaipeda City Municipality, Nepaleisk Svajones.

Renginio after'is: CAPE šiaurinis
Internetinis puslapis: Rocket Science Baltics
Renginio vieta: Lighthouse HUB

Nuotraukos: Artūras Žukas (Arturasz Photography)»

### ChatGPT's Analysis:
1. **Content Quality:** 
   - The post is generally well-written and clear. However, it could benefit from a more professional tone in some areas. For instance, the use of emojis is appropriate, but they might distract from the message in a professional setting.
   - There are no grammatical errors, but some sentences could be more concise.

2. **Reader Engagement:** 
   - The post does a good job of summarizing the event and highlighting speakers, but it lacks direct engagement with the audience. Posing a question or inviting readers to share their thoughts could enhance interaction.

3. **Value Provided:** 
   - The post provides valuable insights into the event and the speakers' contributions. However, it could be more impactful by summarizing key takeaways or lessons learned from the workshops.

4. **Length Analysis:** 
   - The post is on the longer side for LinkedIn. While it contains relevant information, some details could be condensed to maintain reader interest.

5. **LinkedIn Formatting & Readability:** 
   - The formatting could be improved by using bullet points or numbered lists for easier reading. Additionally, breaking up the text into shorter paragraphs would enhance clarity.

### Suggestions:
- Use bullet points for speaker highlights to improve readability.
- Include a call to action to encourage engagement.
- Consider summarizing the key takeaways from the event.
- Reduce the length by removing less critical information.

### Improved Version:
**Praėjusią savaitę Klaipėdoje užbaigėme 3 dalių dirbtuves su Baltic Sandbox, skirtas startuolių kūrėjams ir verslo lyderiams! 🙌**

**Renginyje dalyvavo:**
- **Sandra Golbreich** (BSV Ventures): Kaip supaprastinti sprendimų priėmimą ir transformuoti tradicines verslo struktūras.
- **Mikk Orglaan** (SPARKLY.HR): Startuolių marketingo strategijos ir efektyvus augimo planavimas.
- **Odeta Bloziene** (Beatos Virtuvė), **Žilvinas Kulvinskis** (CHAZZ CHIPS), **Inesa Budgine** (Stand UP Advertising): Atvirai pasidalino (ne)sėkmių patirtimis.

**Dėkojame mūsų partneriams ir rėmėjams už puikų renginį!** 

**Kokie jūsų įspūdžiai apie startuolių ekosistemą? Pasidalinkite komentaruose!**

**Renginio vieta:** Lighthouse HUB  
**Nuotraukos:** Artūras Žukas (Arturasz Photography)  

#startups #business #networking #Klaipeda #BalticSandbox



Processing Post 5/5...



--- Post 5 ---
### Original Post:
«𝗕𝗼𝗻𝗷𝗼𝘂𝗿! 𝗘𝘅𝗽𝗲𝗿𝗶𝗲𝗻𝗰𝗲 𝗕𝗲𝗮𝘂𝘁𝘆 𝗮𝘁 𝗡𝗢𝗖𝗜𝗕É’𝘀 𝗻𝗲𝘄 𝗕𝗼𝘂𝘁𝗶𝗾𝘂𝗲 𝗮𝘁 𝘁𝗵𝗲 𝗚𝗮𝘁𝗲𝘀 𝗼𝗳 𝗣𝗮𝗿𝗶𝘀

As part of its ongoing transformation, NOCIBE – part of the DOUGLAS Group – has unveiled its latest store concept, marking an important step in the rollout of its redefined retail strategy in France. Spanning 320 m², our newest store is built around a clear strategic focus: placing personalized beauty services and evolving customer needs at the core of the in-store experience.

✨ At the heart of the concept: A curated menu of services including facials, spray tans, K-Beauty rituals, a nail bar, and on-site fragrance engraving. The services are available with or without appointment, ensuring accessibility and reinforcing NOCIBÉ’s positioning as a service-led beauty destination.

💆‍♀️ In a more serene space, the store complements this offering with a new treatment menu and “Head Spa” scalp rituals. These additions reflect NOCIBÉ’s strategic alignment with emerging customer needs, particularly the increasing emphasis on the integration of skincare principles into haircare. They also underline the comprehensive approach to well-being and beauty.

📦 Fully integrated with digital services such as Click & Collect Express and Choix+, the new concept enables a seamless omnichannel experience across all touchpoints.

The opening is the first of several store launches planned across France through 2025 – each reinforcing NOCIBÉ’s vision to create experience-driven retail spaces that reflect the evolving expectations of today’s beauty consumers.

Congratulations to the teams at NOCIBÉ and across the DOUGLAS Group on this exciting milestone. 👏

hashtag
#MakeLifeMoreBeautiful»

### ChatGPT's Analysis:
1. **Content Quality:** 
   - The post is generally well-written and professional, with no glaring grammatical errors. However, it could benefit from a more concise structure to enhance clarity.

2. **Reader Engagement:**
   - While the post provides valuable information, it lacks direct engagement with the audience. Posing a question or inviting comments could encourage interaction.

3. **Value Provided:**
   - The post offers insights into NOCIBÉ's new store concept and services, which is valuable. However, it could enhance value by highlighting how these changes specifically benefit the customer.

4. **Length Analysis:**
   - The post is somewhat lengthy for LinkedIn. A more concise version could maintain reader interest while still conveying the essential information.

5. **LinkedIn Formatting & Readability:**
   - The use of emojis and line breaks is effective, but the post could benefit from bullet points for easier scanning. The hashtag is present but could be more relevant to the content.

### Suggestions:
- Shorten sentences for clarity and impact.
- Include a call to action to foster engagement.
- Use bullet points for key features and benefits to enhance readability.
- Consider adding a more relevant hashtag or two to increase discoverability.

### Improved Version:
**🌟 Découvrez la beauté chez NOCIBÉ! 🌟**

Nous sommes ravis d'annoncer l'ouverture de notre nouvelle boutique à Paris, un pas clé dans la transformation de NOCIBÉ, membre du groupe DOUGLAS. 

**✨ Ce qui vous attend :**
- **320 m² d'expérience beauté** : des services personnalisés au cœur de notre concept.
- **Menu de services variés** : soins du visage, bronzage, rituels K-Beauty, bar à ongles et gravure de parfum, disponibles sur rendez-vous ou sans.
- **Nouveau traitement et "Head Spa"** : intégration des principes de soins de la peau dans les soins capillaires.

**📦 Connecté à nos services numériques** : Click & Collect Express et Choix+ pour une expérience omnicanale fluide.

Cette ouverture est la première de plusieurs lancements prévus en France jusqu'en 2025, visant à créer des espaces de vente axés sur l'expérience client.

**Félicitations aux équipes de NOCIBÉ et du groupe DOUGLAS pour cette étape passionnante! 👏**

👉 **Quelles sont vos attentes en matière de services beauté? Partagez vos idées dans les commentaires!**

#NOCIBÉ #Beauté #RetailInnovation #MakeLifeMoreBeautiful



--- Analysis Complete ---

Please enter the desired path to save the output file on Google Drive (e.g., /content/drive/My Drive/linkedin_analysis_results.md): /content/drive/My Drive/linkedin_analysis_results.md

All analysis and improved posts successfully saved to: '/content/drive/My Drive/linkedin_analysis_results.md'
You can find this file in your Google Drive and download it from there.
